In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#!pip3 install numpy
#!pip3 install pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"w
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
complete_train = pd.read_csv('train.csv')
complete_train_og = complete_train.copy()

In [3]:
test = pd.read_csv('kaggle_baseline.csv')

In [4]:
#!pip3 install scikit-learn
from sklearn.preprocessing import MinMaxScaler, StandardScaler


df = complete_train
df.rename(columns={'sex': 'age', 'age': 'sex', 'release_date': 'genre', 'user_id': 'uid', 'movie_id': 'mid'}, inplace=True)
one_hot_genres = df['genre'].str.get_dummies(sep='|')
df = (df.drop(columns=['genre'])).join(one_hot_genres)
df['sex'] = df['sex'].replace({'M': 0, 'F': 1})
df['release_date'] = df['title'].str.extract(r'.*\((\d{4})\).*?$')
df['release_date'] = pd.to_numeric(df['release_date'], errors='coerce')
df['release_date_rank'] = df['release_date'].rank(method='average')
# Initialize a MinMaxScaler
scaler = MinMaxScaler()
# Rescale the ranked release dates to between 0 and 1
df['normalized_release_date'] = scaler.fit_transform(df['release_date_rank'].values.reshape(-1, 1))

unique_ages_sorted = sorted(df['age'].unique())

# Map each age to an equidistant point between 0 and 1
age_to_normalized = {age: i / (len(unique_ages_sorted) - 1) for i, age in enumerate(unique_ages_sorted)}

# Apply mapping to the original age values
df['normalized_age'] = df['age'].map(age_to_normalized)


df = df.drop(['title', 'age', 'release_date'	,'release_date_rank'], axis=1)
df

/var/folders/yb/mm0hrmlx3g9dv10c735753m00000gn/T/ipykernel_43317/3025904736.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sex'] = df['sex'].replace({'M': 0, 'F': 1})


,uid,mid,rating,sex,Action,Adventure,Animation,Children's,Comedy,Crime,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,normalized_release_date,normalized_age
0,2592,1101,4,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0.341005,0.833333
1,4318,1203,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.061125,0.333333
2,2756,2986,2,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0.450953,0.166667
3,1706,3462,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.007844,0.333333
4,4813,276,3,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0.604616,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800162,59,2352,4,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.265322,0.833333
800163,4458,543,4,1,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0.554112,0.333333
800164,1234,3897,4,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1.000000,0.166667
800165,4864,1079,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.394491,0.166667


In [5]:
def standardize_and_normalize(group):
    # Standardize
    mean_rating = group['rating'].mean()
    std_rating = group['rating'].std()
    if std_rating > 0:
        group['standardized_rating'] = (group['rating'] - mean_rating) / std_rating
    else:
        group['standardized_rating'] = 0.0  # Use float for consistency

    # Normalize
    min_std_rating = group['standardized_rating'].min()
    max_std_rating = group['standardized_rating'].max()
    range_std_rating = max_std_rating - min_std_rating
    if range_std_rating > 0:
        group['norm_std_rating'] = (group['standardized_rating'] - min_std_rating) / range_std_rating
    else:
        group['norm_std_rating'] = 0.5  # Use 0.5 to represent middle of the range if no variation

    return group

def set_top_scores(group):
    # Find the highest rating
    highest_rating = group['rating'].max()
    if highest_rating == 5: # or highest_rating == 4:
        group['new_rating'] = group['rating'].apply(lambda x: 1 if x == highest_rating or x == highest_rating - 1 else 0)
    else:
        group['new_rating'] = group['rating'].apply(lambda x: 1 if x == highest_rating else 0)        
    
    return group




# Apply the function to each user's set of ratings
#df = df.groupby('uid').apply(standardize_and_normalize).reset_index(drop=True)
#df = df.drop(['standardized_rating', 'rating'], axis=1)
#df.rename(columns={ 'norm_std_rating': 'rating'}, inplace=True)

df = df.groupby('uid').apply(set_top_scores).reset_index(drop=True)
df = df.drop(['rating'], axis=1)  # Remove original rating column
df.rename(columns={'new_rating': 'rating'}, inplace=True)



df.rename(columns={'normalized_release_date': 'release_year', 'normalized_age': 'age'}, inplace=True)
df

/var/folders/yb/mm0hrmlx3g9dv10c735753m00000gn/T/ipykernel_43317/1681492736.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('uid').apply(set_top_scores).reset_index(drop=True)


,uid,mid,sex,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,release_year,age,rating
0,1,1566,1,0,1,1,1,1,0,0,...,1,0,0,0,0,0,0,0.787452,0.000000,1
1,1,3408,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.000000,0.000000,1
2,1,1,1,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0.662515,0.000000,1
3,1,1721,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0.787452,0.000000,1
4,1,1193,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.164443,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800162,6040,1305,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.286243,0.333333,1
800163,6040,3359,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.199742,0.333333,1
800164,6040,2391,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0.855509,0.333333,1
800165,6040,722,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.662515,0.333333,0


In [6]:
genres_columns = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
# Create lookup tables for ages, sex, and release_year
user_ages_lookup = df[['uid', 'age']].drop_duplicates().set_index('uid')['age'].to_dict()
user_sex_lookup = df[['uid', 'sex']].drop_duplicates().set_index('uid')['sex'].to_dict()
movie_release_year_lookup = df[['mid', 'release_year']].drop_duplicates().set_index('mid')['release_year'].to_dict()
movie_genres_lookup = df[['mid'] + genres_columns].drop_duplicates().set_index('mid')[genres_columns].to_dict('index')

# Generate all possible user-movie pairs for the extended implicit feedback
uids = df['uid'].unique()
mids = df['mid'].unique()
um_pairs = pd.DataFrame([(uid, mid) for uid in uids for mid in mids], columns=['uid', 'mid'])

um_pairs = um_pairs.merge(df[['uid', 'mid', 'rating']], on=['uid', 'mid'], how='left')
um_pairs['rating'].fillna(0, inplace=True)

um_pairs['age'] = um_pairs['uid'].map(user_ages_lookup).fillna(0)
um_pairs['sex'] = um_pairs['uid'].map(user_sex_lookup).fillna(0) 
um_pairs['release_year'] = um_pairs['mid'].map(movie_release_year_lookup).fillna(0)

for genre in genres_columns:
   um_pairs[genre] = um_pairs['mid'].map(lambda mid: movie_genres_lookup[mid][genre] if mid in movie_genres_lookup else 0)

um_pairs

/var/folders/yb/mm0hrmlx3g9dv10c735753m00000gn/T/ipykernel_43317/2476428434.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  um_pairs['rating'].fillna(0, inplace=True)


,uid,mid,rating,age,sex,release_year,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1566,1.0,0.000000,1,0.787452,0,1,1,1,...,0,0,0,1,0,0,0,0,0,0
1,1,3408,1.0,0.000000,1,1.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1.0,0.000000,1,0.662515,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1721,1.0,0.000000,1,0.787452,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,1193,1.0,0.000000,1,0.164443,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22227195,6040,2703,0.0,0.333333,0,0.855509,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22227196,6040,2845,0.0,0.333333,0,0.934641,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22227197,6040,3607,0.0,0.333333,0,0.145098,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22227198,6040,1360,0.0,0.333333,0,0.248562,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
ready_um_pairs = um_pairs

In [8]:
same_sex = um_pairs.groupby('mid')['Action'].nunique() == 1
all_same_sex = same_sex.all()
print(all_same_sex)


True


## Baseline Recommender

In [9]:
class TopPopRecommender():

    def fit(self, train):

        item_popularity = train[['movie_id','rating']].groupby(by='movie_id').count()

        self.train = train
        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = item_popularity.sort_values(by='rating',ascending=False).index


    def predict_top(self, user_id, at=5, remove_seen=True):

        if remove_seen:
            seen_items = self.train[self.train.user_id==user_id].movie_id.values
            unseen_items_mask = np.in1d(self.popular_items, seen_items, assume_unique=True, invert = True)
            unseen_items = self.popular_items[unseen_items_mask]
            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popular_items[0:at]

        return recommended_items

## Attempt 1

In [28]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Reshape, Lambda, Add, Concatenate, Multiply, Subtract
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow.keras.backend as Kl
from tensorflow.keras.layers import Input, Dense, Embedding, Reshape, Lambda, Add, Concatenate, Multiply, Subtract,  Flatten, Dropout



def tensor_sum(name='tensor_sum'):
    return Lambda(lambda x: K.sum(x, axis=1, keepdims=True), name=name)

def tensor_square(name='tensor_square'):
    return Lambda(lambda x: K.square(x), name=name)


class MovieRecommender4FM1:
    def __init__(self, n_uid, n_mid, k):
        self.n_uid = n_uid
        self.n_mid = n_mid
        self.k = k
        self.topPopular = TopPopRecommender()
        self.model = self._create_model()

    def fm_1d(self, uid_input, mid_input, sex_input, release_year_input, age_input, genres_input, n_uid, n_mid):
        uid_embed_1d = Flatten()(Embedding(n_uid + 1, 1)(uid_input))
        mid_embed_1d = Flatten()(Embedding(n_mid + 1, 1)(mid_input))
        sex_dense = Dense(1, use_bias=False)(sex_input)
        release_year_dense = Dense(1, use_bias=False)(release_year_input)
        age_dense = Dense(1, use_bias=False)(age_input)
        linear_terms = Add()([uid_embed_1d, mid_embed_1d, sex_dense, release_year_dense, age_dense, genres_input])
        return linear_terms

    def fm_2d(self, uid_input, mid_input, sex_input, release_year_input, age_input, genres_input, n_uid, n_mid, k):
        uid_embed_2d = Flatten()(Embedding(n_uid + 1, k)(uid_input))
        mid_embed_2d = Flatten()(Embedding(n_mid + 1, k)(mid_input))
        sex_embed_2d = Dense(k, use_bias=False)(sex_input)
        release_year_embed_2d = Dense(k, use_bias=False)(release_year_input)
        age_embed_2d = Dense(k, use_bias=False)(age_input)
        concatenated_features = Concatenate()([uid_embed_2d, mid_embed_2d, sex_embed_2d, release_year_embed_2d, age_embed_2d, genres_input])
        squared_sum_features = Lambda(lambda x: K.square(K.sum(x, axis=1, keepdims=True)))(concatenated_features)
        sum_squared_features = Lambda(lambda x: K.sum(K.square(x), axis=1, keepdims=True))(concatenated_features)
        interaction_term = Subtract()([squared_sum_features, sum_squared_features])
        return interaction_term

    def _create_model(self):
        uid_input = Input(shape=(1,), dtype='int32', name='uid_input')
        mid_input = Input(shape=(1,), dtype='int32', name='mid_input')
        sex_input = Input(shape=(1,), name='sex_input')
        release_year_input = Input(shape=(1,), name='release_year_input')
        age_input = Input(shape=(1,), name='age_input')
        genres_input = Input(shape=(18,), name='genres_input')  # Assuming 18 genres
        fm_1d_output = self.fm_1d(uid_input, mid_input, sex_input, release_year_input, age_input, genres_input, self.n_uid, self.n_mid)
        fm_2d_output = self.fm_2d(uid_input, mid_input, sex_input, release_year_input, age_input, genres_input, self.n_uid, self.n_mid, self.k)
        combined_output = Add()([fm_1d_output, Flatten()(fm_2d_output)])
        final_output = Dense(1, activation='sigmoid')(combined_output)
        model = Model(inputs=[uid_input, mid_input, sex_input, release_year_input, age_input, genres_input], outputs=final_output)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model


    def df2xy_and_split(self, df, test_size=0.2):
        um_pairs = df
        X = um_pairs[['uid', 'mid', 'sex', 'release_year', 'age'] + genres_columns].values
        y = um_pairs['rating'].values

        train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=42)
        
        train_features = [train_x[:, i].reshape(-1, 1) for i in range(5)]
        train_features.append(train_x[:, 5:])
        
        test_features = [test_x[:, i].reshape(-1, 1) for i in range(5)]
        test_features.append(test_x[:, 5:])
        
        return train_features, train_y, test_features, test_y

    def fit(self, df, pop_source, epochs=30, batch_size=2048, patience=3, kaggle_mode=False):
        self.topPopular.fit(pop_source)
        if kaggle_mode:
            train_x, train_y, test_x, test_y = self.df2xy_and_split(df, test_size=0.00001)
            #train_x, train_y, test_x, test_y = self.df2xy_and_split(df)
        else:
            train_x, train_y, test_x, test_y = self.df2xy_and_split(df)
            self.valid_x = test_x
            self.valid_y = test_y     
        
        early_stop = EarlyStopping(monitor='val_loss', patience=patience)
        model_ckp = ModelCheckpoint(filepath='deepfm_.weights.h5', monitor='val_loss', save_weights_only=True, save_best_only=True)
        callbacks = [model_ckp, early_stop]

        train_history = self.model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,validation_split=0.1, callbacks=callbacks)#validation_data=(valid_x, valid_y), 
        return train_history

    def evaluate_model(self):
        #valid_x, valid_y = self.df2xy(val_df)
        predictions = self.model.predict(self.valid_x)
        rmse = np.sqrt(mean_squared_error(self.valid_y, predictions))
        print("MSE:", mean_squared_error(self.valid_y, predictions))
        print("RMSE:", rmse)
    
    def recommend_movies(self, user_id, top_n=25):
            # Assuming self.user_ages_lookup, self.user_sex_lookup, self.movie_release_year_lookup, 
            # and self.movie_genres_lookup are already populated
            
            # Extract genres columns names as used in earlier steps
            genres_columns = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary',
                            'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                            'Thriller', 'War', 'Western']
            
            # Fetch the age and sex for the user
            user_age = user_ages_lookup.get(user_id, 0)  # Default 0 if user_id is not found
            user_sex = user_sex_lookup.get(user_id, 0)  # Default 0 if user_id is not found
            
            # Generate all possible unseen movie IDs using the topPopular recommender
            unseen_movie_ids = np.array(self.topPopular.predict_top(user_id, at=200, remove_seen=True))
            
            # Prepare the prediction input arrays
            sex_array = np.full((len(unseen_movie_ids), 1), fill_value=user_sex)
            age_array = np.full((len(unseen_movie_ids), 1), fill_value=user_age)
            genres_array = np.array([[genre_dict.get(genre, 0) for genre in genres_columns] 
                            if (genre_dict := movie_genres_lookup.get(mid)) is not None 
                            else np.zeros(len(genres_columns)) for mid in unseen_movie_ids])
            release_year_array = np.array([movie_release_year_lookup.get(mid, 0) for mid in unseen_movie_ids])
            user_ids_array = np.full((len(unseen_movie_ids), ), user_id)
            movie_ids_array = unseen_movie_ids  # Ensure it's in a format that supports reshape
            
            # Prepare the data in the correct format, incorporating all features
            predict_x = [user_ids_array.reshape(-1, 1), movie_ids_array.reshape(-1, 1), 
                        sex_array, release_year_array.reshape(-1, 1), age_array, genres_array]
            
            # Predict the ratings for all movies for the given user_id
            predicted_ratings = self.model.predict(predict_x).flatten()
            
            # Sort the predictions and select top N
            top_n_movie_indices = np.argsort(-predicted_ratings)[:top_n]
            top_n_movie_ids = unseen_movie_ids[top_n_movie_indices]

            return top_n_movie_ids



In [29]:

from sklearn.model_selection import train_test_split

genres = ["Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
n_uid = ready_um_pairs['uid'].nunique()
n_mid = ready_um_pairs['mid'].nunique()
k = 20
#train_df, test_df = train_test_split(ready_df, test_size=0.2)


model1 = MovieRecommender4FM1(n_uid, n_mid, k)
model1.fit(ready_um_pairs, complete_train_og, epochs=30, batch_size=2048)
#print("IDs der 25 größten Werte:", model4.recommend_movies(1))
#model6.recommend_movies(1)
model1.evaluate_model()

Epoch 1/30
7815/7815 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - loss: 0.0382 - val_loss: 0.0179
Epoch 2/30
7815/7815 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.0177 - val_loss: 0.0178
Epoch 3/30
7815/7815 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.0177 - val_loss: 0.0178
Epoch 4/30
7815/7815 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.0176 - val_loss: 0.0178
Epoch 5/30
7815/7815 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.0177 - val_loss: 0.0178
Epoch 6/30
7815/7815 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.0177 - val_loss: 0.0178
Epoch 7/30
7815/7815 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - loss: 0.0176 - val_loss: 0.0178
Epoch 8/30
7815/7815 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.0176 - val_loss: 0.0178
138920/138920 ━━━━━━━━━━━━━━━━━━━━ 60s 431us/step
MSE: 0.017697223078078025
RMSE: 0.1330309102354713


# Queeee????



In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Reshape, Lambda, Add, Concatenate, Multiply, Subtract
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


def tensor_sum(name='tensor_sum'):
    return Lambda(lambda x: K.sum(x, axis=1, keepdims=True), name=name)

def tensor_square(name='tensor_square'):
    return Lambda(lambda x: K.square(x), name=name)


class MovieRecommenderFM5:
    def __init__(self, n_uid, n_mid, k):
        self.topPopular = TopPopRecommender()
        self.n_uid = n_uid
        self.n_mid = n_mid
        self.k = k
        self.model = self._create_model()

    def fm_1d(self, uid_input, mid_input, age_input, genres_input, n_uid, n_mid):
        # Embedding layers for uid and mid, each followed by a Flatten layer
        uid_embed_1d = Flatten()(Embedding(n_uid + 1, 1, name='uid_embed_1d')(uid_input))
        mid_embed_1d = Flatten()(Embedding(n_mid + 1, 1, name='mid_embed_1d')(mid_input))
        
        # Dense layer for the age input
        age_dense = Dense(1, use_bias=False, name='age_dense')(age_input)
        
        # Combining all the 1D contributions including genres_input directly for additive model part
        linear_terms = Add()([uid_embed_1d, mid_embed_1d, age_dense, genres_input])
        return linear_terms

    def fm_2d(self, uid_input, mid_input, age_input, genres_input, n_uid, n_mid, k):
        # Flatten embedding layers to fit the expected 2D interaction tensor shape
        uid_embed_2d = Flatten()(Embedding(n_uid + 1, k, name='uid_embed_2d')(uid_input))
        mid_embed_2d = Flatten()(Embedding(n_mid + 1, k, name='mid_embed_2d')(mid_input))
        
        # Dense layer to transform age input into a k-dimensional vector for interaction
        age_embed_2d = Dense(k, use_bias=False, name='age_embed_2d')(age_input)
        
        # Concatenate all features for 2D interactions
        concatenated_features = Concatenate()([uid_embed_2d, mid_embed_2d, age_embed_2d, genres_input])
        
        # Compute the squared sum of features for the 2D interactions
        squared_sum_features = Lambda(lambda x: K.square(K.sum(x, axis=1, keepdims=True)))(concatenated_features)
        
        # Compute the sum of squared features for the 2D interactions
        sum_squared_features = Lambda(lambda x: K.sum(K.square(x), axis=1, keepdims=True))(concatenated_features)
        
        # Interaction term by subtracting the squared_sum_features from sum_squared_features
        interaction_term = Subtract()([squared_sum_features, sum_squared_features])
        
        return interaction_term

    def _create_model(self):
        # Define inputs
        uid_input = Input(shape=(1,), dtype='int32', name='uid_input')
        mid_input = Input(shape=(1,), dtype='int32', name='mid_input')
        age_input = Input(shape=(1,), name='age_input')
        genres_input = Input(shape=(18,), name='genres_input')  # Assuming 18 genres are one-hot encoded
        
        # Calculate outputs from fm_1d and fm_2d functions
        fm_1d_output = self.fm_1d(uid_input, mid_input, age_input, genres_input, self.n_uid, self.n_mid)
        fm_2d_output = self.fm_2d(uid_input, mid_input, age_input, genres_input, self.n_uid, self.n_mid, self.k)
        
        # Combine 1D and 2D interaction outputs
        combined_output = Add()([fm_1d_output, Flatten()(fm_2d_output)])
        
        # Final output predictions
        final_output = Dense(1, activation='sigmoid')(combined_output)
        
        # Model compilation
        model = Model(inputs=[uid_input, mid_input, age_input, genres_input], outputs=final_output)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        
        return model

    def df2xy_and_split(self, df, test_size=0.2):
            # Assuming 'df' has user IDs, movie IDs, ratings, user ages, and movie genres.

            # Create lookup tables for ages and genres
            self.user_ages_lookup = df[['uid', 'age']].drop_duplicates().set_index('uid')['age'].to_dict()
            genres_columns = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary',
                            'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                            'Thriller', 'War', 'Western']
            self.movie_genres_lookup = df[['mid'] + genres_columns].drop_duplicates().set_index('mid')[genres_columns].to_dict('index')

            # Generate all possible user-movie pairs
            uids = df['uid'].unique()
            mids = df['mid'].unique()
            um_pairs = pd.DataFrame([(uid, mid) for uid in uids for mid in mids], columns=['uid', 'mid'])

            # Merge existing ratings
            um_pairs = um_pairs.merge(df[['uid', 'mid', 'rating']], on=['uid', 'mid'], how='left')
            um_pairs['rating'].fillna(0, inplace=True)  # Fill missing ratings with 0

            # Populate ages and genres using the lookup tables
            um_pairs['age'] = um_pairs['uid'].map(self.user_ages_lookup).fillna(0)  # Filling missing ages just in case, though there shouldn't be any
            for genre in genres_columns:
                um_pairs[genre] = um_pairs['mid'].map(lambda mid: self.movie_genres_lookup[mid][genre] if mid in self.movie_genres_lookup else 0)

            # Preparing the final dataset for model
            X = um_pairs[['uid', 'mid', 'age'] + genres_columns].values
            y = um_pairs['rating'].values

            # Splitting the data into training and testing sets
            train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size)

            # Splitting the features back into separate arrays for the model
            train_x = [train_x[:, 0].reshape(-1, 1), train_x[:, 1].reshape(-1, 1), train_x[:, 2].reshape(-1, 1), train_x[:, 3:]]
            test_x = [test_x[:, 0].reshape(-1, 1), test_x[:, 1].reshape(-1, 1), test_x[:, 2].reshape(-1, 1), test_x[:, 3:]]

            return train_x, train_y, test_x, test_y

    
    def df2xy(self, df):
        # Assuming 'df' contains columns 'uid', 'mid', and 'rating'
        # Also, it is assumed that the class has 'user_ages_lookup' and 'movie_genres_lookup' already computed
        self.user_ages_lookup = df[['uid', 'age']].drop_duplicates().set_index('uid')['age'].to_dict()
        genres_columns = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary',
                                'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                                'Thriller', 'War', 'Western']
        self.movie_genres_lookup = df[['mid'] + genres_columns].drop_duplicates().set_index('mid')[genres_columns].to_dict('index')
        # Find all unique user IDs and movie IDs in 'df' to create a full dataset
        unique_uids = df['uid'].unique()
        unique_mids = df['mid'].unique()
        
        # Create a DataFrame with all combinations of 'uid' and 'mid' to ensure coverage of all possible interactions
        um_pairs = pd.DataFrame([(uid, mid) for uid in unique_uids for mid in unique_mids], columns=['uid', 'mid'])

        # Merge with the original 'df' to get ratings where they exist
        um_pairs = um_pairs.merge(df[['uid', 'mid', 'rating']], on=['uid', 'mid'], how='left')

        # Fill missing ratings with 0 to indicate no interaction
        um_pairs['rating'].fillna(0, inplace=True)
        
        # Map user ages and genres to each user-movie pair
        um_pairs['age'] = um_pairs['uid'].map(self.user_ages_lookup).fillna(0)

        # Prepare genres columns names
        genres_columns = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary',
                        'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                        'Thriller', 'War', 'Western']
        
        # For each genre column, map movie genres based on 'mid' using the lookup table
        for genre in genres_columns:
            um_pairs[genre] = um_pairs['mid'].apply(lambda x: self.movie_genres_lookup[x][genre] if x in self.movie_genres_lookup else 0)
        
        # Extract the target variable 'rating'
        y = um_pairs['rating'].values
        
        # Preparing the feature arrays
        x = [
            um_pairs['uid'].values.reshape(-1, 1),  # User ID
            um_pairs['mid'].values.reshape(-1, 1),  # Movie ID
            um_pairs['age'].values.reshape(-1, 1),  # User Age
            um_pairs[genres_columns].values         # Movie Genres
        ]
        
        return x, y

    def fit(self, df, pop_source, epochs=30, batch_size=2048, patience=3, kaggle_mode=False):
        self.topPopular.fit(pop_source)
        if kaggle_mode:
            train_x, train_y, test_x, test_y = self.df2xy_and_split(df, test_size=0.00001)
            #train_x, train_y, test_x, test_y = self.df2xy_and_split(df)
        else:
            train_x, train_y, test_x, test_y = self.df2xy_and_split(df)
            self.valid_x = test_x
            self.valid_y = test_y     
        
        early_stop = EarlyStopping(monitor='val_loss', patience=patience)
        model_ckp = ModelCheckpoint(filepath='deepfm_.weights.h5', monitor='val_loss', save_weights_only=True, save_best_only=True)
        callbacks = [model_ckp, early_stop]

        train_history = self.model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,validation_split=0.1, callbacks=callbacks)#validation_data=(valid_x, valid_y), 
        return train_history

    def evaluate_model(self):
        #valid_x, valid_y = self.df2xy(val_df)
        predictions = self.model.predict(self.valid_x)
        rmse = np.sqrt(mean_squared_error(self.valid_y, predictions))
        print("MSE:", mean_squared_error(self.valid_y, predictions))
        print("RMSE:", rmse)
    
    def recommend_movies(self, user_id, top_n=25):
        # Assuming self.user_ages_lookup and self.movie_genres_lookup are already populated
            
        # Extract genres columns names as used in earlier steps
        genres_columns = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary',
                    'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                    'Thriller', 'War', 'Western']

        # Fetch the age for the user
        user_age = self.user_ages_lookup.get(user_id, 0)  # Default 0 if user_id is not found
        
        # Generate all possible unseen movie IDs, assuming self.topPopular is correctly implemented to return such list
        # Here we make sure to convert it into a NumPy array to support the reshape operation later
        unseen_movie_ids = np.array(self.topPopular.predict_top(user_id, at=200, remove_seen=True))
        
        # Prepare the prediction input arrays
        age_array = np.full((len(unseen_movie_ids), 1), fill_value=user_age)
        genres_array = np.array([[genre_dict.get(genre, 0) for genre in genres_columns] 
                     if (genre_dict := self.movie_genres_lookup.get(mid)) is not None 
                     else np.zeros(len(genres_columns)) for mid in unseen_movie_ids])
        user_ids_array = np.full((len(unseen_movie_ids), ), user_id)
        movie_ids_array = unseen_movie_ids  # Now ensure it's in a format that supports reshape
        
        # Prepare the data in the correct format
        predict_x = [user_ids_array.reshape(-1, 1), movie_ids_array.reshape(-1, 1), age_array, genres_array]
        
        # Predict the ratings for all movies for the given user_id
        predicted_ratings = self.model.predict(predict_x).flatten()
        
        # Sort the predictions and select top N
        top_n_movie_indices = np.argsort(-predicted_ratings)[:top_n]
        top_n_movie_ids = unseen_movie_ids[top_n_movie_indices]

        return top_n_movie_ids



    


In [12]:

from sklearn.model_selection import train_test_split

genres = ["Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
n_uid = df['uid'].nunique()
n_mid = df['mid'].nunique()
k = 20
#train_df, test_df = train_test_split(ready_df, test_size=0.2)


model5 = MovieRecommenderFM5(n_uid, n_mid, k)
model5.fit(df, complete_train_og, epochs=30, batch_size=2048)
#print("IDs der 25 größten Werte:", model4.recommend_movies(1))
#model6.recommend_movies(1)
model5.evaluate_model()

KeyboardInterrupt: 